In [4]:
import pandas as pd 

data = pd.read_csv(r'C:\Users\vinee\OneDrive\Desktop\projects\BondYeild_Prediction\newfredgraph.csv')
print(data.head())

         DATE    FF  INDPRO   CPI    VIX  DGS10
0  01-01-2000  5.45    4.80  2.11  23.20   6.66
1  01-02-2000  5.73    4.56  2.16  23.60   6.52
2  01-03-2000  5.85    4.74  2.45  22.72   6.26
3  01-04-2000  6.02    5.17  2.27  27.16   5.99
4  01-05-2000  6.27    4.84  2.38  26.37   6.44


In [5]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    296 non-null    object 
 1   FF      296 non-null    float64
 2   INDPRO  296 non-null    float64
 3   CPI     296 non-null    float64
 4   VIX     296 non-null    float64
 5   DGS10   296 non-null    float64
dtypes: float64(5), object(1)
memory usage: 14.0+ KB
None


In [6]:
print(data.describe())

               FF      INDPRO         CPI         VIX       DGS10
count  296.000000  296.000000  296.000000  296.000000  296.000000
mean     1.881081    0.681655    2.378851   19.905541    3.272027
std      2.008133    4.405529    1.142222    8.118895    1.306952
min      0.050000  -17.200000    0.600000   10.130000    0.620000
25%      0.140000   -1.062500    1.740000   14.077500    2.197500
50%      1.160000    1.900000    2.140000   17.820000    3.290000
75%      3.072500    3.090000    2.482500   23.457500    4.262500
max      6.540000   16.120000    6.640000   62.670000    6.660000


In [7]:
correlation_matrix = data.drop(columns= ['DATE']).corr()
print(correlation_matrix)

              FF    INDPRO       CPI       VIX     DGS10
FF      1.000000  0.134926  0.347424 -0.139731  0.752793
INDPRO  0.134926  1.000000  0.143670 -0.434660  0.148817
CPI     0.347424  0.143670  1.000000  0.026629  0.085390
VIX    -0.139731 -0.434660  0.026629  1.000000 -0.004733
DGS10   0.752793  0.148817  0.085390 -0.004733  1.000000


In [13]:
!pip install scikit-learn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ----------------- ---------------------- 4.7/11.0 MB 26.4 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 30.0 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 22.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ------ --------------------------------- 7.6/44.5 MB 35.5 MB/s eta 0:00:02
   ------------ --------------------------- 14.2/44.5 MB 33.6 MB/s eta 0:00:01
   --------------------- ------------------ 23.6/44.5 MB 37.6 MB/s eta 0:00:01
   ---------------------------- ----------- 31.7/44.5 MB 37.8 MB/s eta 0:00:01
   ---------------------------------- ----- 38.3/44.5 MB 37.0 MB/s eta 0:00:01
   ---------------------------------------  44.3/44.5 MB 37.1 MB/s eta 0:00:01
   ---------------------------------------- 44.5/44.5 MB 32.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
#normalizing the input variables
from sklearn.preprocessing import MinMaxScaler

data['DATE'] = pd.to_datetime(data['DATE'], format='%d-%m-%Y')

X= data[['FF','INDPRO','CPI','VIX']]
Y= data[['DGS10']]

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

#Keeping the column names, converting back to dataframe
X_scaled_DF = pd.DataFrame(X_scaled, columns=X.columns)

#combining the data
N_data = pd.concat([data['DATE'], X_scaled_DF,Y], axis =1)

print(N_data.head())

N_data.to_csv('normalized_bonddata.csv', index = False)


        DATE        FF    INDPRO       CPI       VIX  DGS10
0 2000-01-01  0.832049  0.660264  0.250000  0.248763   6.66
1 2000-02-01  0.875193  0.653061  0.258278  0.256376   6.52
2 2000-03-01  0.893683  0.658463  0.306291  0.239627   6.26
3 2000-04-01  0.919877  0.671369  0.276490  0.324134   5.99
4 2000-05-01  0.958398  0.661465  0.294702  0.309098   6.44


In [15]:
print(N_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   DATE    296 non-null    datetime64[ns]
 1   FF      296 non-null    float64       
 2   INDPRO  296 non-null    float64       
 3   CPI     296 non-null    float64       
 4   VIX     296 non-null    float64       
 5   DGS10   296 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 14.0 KB
None


In [22]:
!python -m pip install --upgrade pip

In [20]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [ ]:
#splitting the dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tesnorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

data = pd.read_csv(r'C:\Users\vinee\OneDrive\Desktop\projects\BondYeild_Prediction\normalized_bonddata.csv')

#splitting the data into training(2000-2020) and testing (2020-2024)
train_data = data[(data['DATE']>= '2000-01-01') & (data['DATE'] <= '2020-12-31')]
test_data = data[(data['DATE']>= '2020-01-01') & (data['DATE'] <= '2024-12-31')]

# seperating features and target
X_train = train_data[['FF','CPI','INDPRO','VIX']]
Y_train = train_data['DGS10']

X_test = test_data[['FF','CPI','INDPRO','VIX']]
Y_test = test_data['DGS10']

MODEL BUILDING

In [16]:
model = Sequential()
#input layer
model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)))
#second layer
model.add(Dense(units=32, activation='relu'))
#third layer
model.add(Dense(units=16, activation='relu'))

#output layer
model.add(Dense(units=1))

model.compile(optimizer=Adam(learning_rate = 0.0001), loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

#Predictions
Y_pred = model.predict(X_test)

#comparing the yields
comparison = pd.DataFrame({'Date': test_data['DATE'], 'Actual': Y_test,'Predicted':Y_pred.flatten()})
print(comparison)

#evaluating the model
loss = model.evaluate(X_test, Y_test)
print(f'Mean Squared Error on the test set: {loss}')


NameError: name 'Sequential' is not defined